# 要約 
このJupyter Notebookでは、「20の質問」ゲームに特化した言語モデルを実装するためのエージェントを構築しています。具体的には、質問者エージェント（`Llama3QuestionerAgent`）と回答者エージェント（`Llama3AnswererAgent`）の2つのエージェントを作成し、ゲームルールに従って対話を行えるようにします。

### 問題の取り組み
- **目的**: 20の質問ゲームをプレイするためのAIエージェントを開発し、ターゲットワードに関連する質問を生成し、回答を提供する。
- **ゲームのルール**: 質問者エージェントがターゲットワードに対して質問をし、回答者エージェントが「はい」または「いいえ」で応答します。この両者の協力によって、できるだけ早くターゲットを特定することを目指します。

### 手法とライブラリ
1. **Transformersライブラリ**:
   - `AutoTokenizer`と`AutoModelForCausalLM`を用いて、事前訓練された言語モデルをロードします。具体的には、LLAMA-3モデルを使用して、そのプロンプトや出力形式を管理します。
2. **PyTorch**:
   - モデルの生成時には、PyTorchを利用して効率的に計算を実行します。特に、メモリ効率を高めるためにSDP（システムデータパラメータ）を調整します。
3. **状態管理**:
   - `Llama3Formatter`クラスを使って、ユーザーの入力やモデルの出力、システムのプロンプトを状態として管理し、ゲームの流れをスムーズにします。

### エージェントの構成
- **質問者エージェント**:
  - ゲームの進行に従い、対象の単語が「場所」であるかを特定したり、単語の最初や二文字目の範囲を絞り込んだりします。
- **回答者エージェント**:
  - 質問に対して「はい」または「いいえ」といった応答をし、ターゲットの単語に対する情報を提供します。

### その他の構成要素
- **出力の保存**: 最後に、生成したエージェントのコードを一つのtar.gzファイルにアーカイブし、次のステップでの提出に備えるために整理しています。
- **エラーハンドリング**: 応答が無効な場合にはデフォルトの「はい」を返すなど、冗長性のある設計がなされています。

このノートの内容は、20の質問ゲームを自動化し、AIが協力して効率的にターゲットを推測する能力を示すものとなっています。

---


# 用語概説 
以下に、Jupyter Notebook内で使用されている専門用語の簡単な解説を示します。初心者でもある程度の知識がある前提のため、深く掘り下げた説明は省いています。

1. **KAGGLE_AGENT_PATH**:
   - Kaggleのシミュレーション環境内でエージェントのライブラリが格納されているパスを指定します。このパスを使うことで、ノートブックとシミュレーション環境でコードが正しく動作するように設定します。

2. **トークナイザー (Tokenizer)**:
   - テキストデータをモデルが理解可能なトークン（単語やサブワードなどの単位）に変換するためのツールです。モデルの入力として使用されるため、重要です。

3. **Causal LM (因果言語モデル)**:
   - 与えられたテキストの文脈に基づいて次の単語を予測するためのモデル。通常の言語モデルは全トークンを考慮しますが、因果モデルはこれに制約があり、現在のトークンより前のトークンのみを考慮します。

4. **メモリ効率の良いSDP** (Memory Efficient SDP):
   - メモリ効率を高めるための技術で、大規模な深層学習モデルのトレーニングや推論を行う際に必要とされるメモリの使用を削減します。

5. **bfloat16**:
   - 16ビットのブレイン浮動小数点形式で、深層学習においてメモリ使用量を抑えつつ、計算精度を維持するために用いられるデータ型です。

6. **プロンプト (Prompt)**:
   - モデルに対して提供する入力のこと。例えば質問やテーマなどを含み、モデルがどのように応答するかを決定づけます。

7. **Few-shot examples**:
   - モデルに少数の参考例（少数ショット例）を与え、特定のタスクにおいてどのように振る舞うべきかを学習させる手法。例を基にモデルが自己修正を行いやすくなります。

8. **エージェント (Agent)**:
   - ある特定のタスクを実行するために設計されたプログラムやモデルのこと。ここでは「質問者」や「回答者」としての特定のロールを持つエージェントが扱われています。

9. **interleave_unequal**:
   - 異なる長さのリストを交互に組み合わせるための関数。異なる情報源からの情報を組み合わせることにより、より効率的なデータ処理を行います。

10. **Top-pサンプリング**:
    - 確率分布に基づいて上位pの確率質量を持つ単語の中から新しいトークンをサンプリングする手法。生成する内容に多様性を持たせます。

これらの用語は、このノートブックの理解や、機械学習・深層学習の実践において特に重要です。

---


In [ ]:
%%bash
# 新しいディレクトリを作成します
# -pオプションは、親ディレクトリも必要に応じて作成します
mkdir -p /kaggle/working/submission

In [ ]:
%%writefile submission/main.py
# 設定
import os
import sys

# **重要:** このようにシステムパスを設定して、ノートブックとシミュレーション環境の両方でコードが正しく動作するようにします。
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")

# Transformersライブラリからトークナイザーとモデルをインポート
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# https://github.com/Lightning-AI/litgpt/issues/327
torch.backends.cuda.enable_mem_efficient_sdp(False) # メモリ効率の良いSDPを無効に
torch.backends.cuda.enable_flash_sdp(False) # フラッシュSDPを無効に

if os.path.exists(KAGGLE_AGENT_PATH):
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "llama-3/transformers/8b-chat-hf/1")
else:
    WEIGHTS_PATH = "/kaggle/input/llama-3/transformers/8b-chat-hf/1"

# プロンプトのフォーマット
import itertools
from typing import Iterable


class Llama3Formatter:
    def __init__(self, system_prompt: str = None, few_shot_examples: Iterable = None):
        self._system_prompt = system_prompt
        self._few_shot_examples = few_shot_examples
        self.reset()  # 初期状態をリセット

    def get_dict(self):
        return self._state  # 現在の状態を辞書形式で取得

    def user(self, prompt):
        self._state.append({'role': 'user', 'content': prompt})  # ユーザーの入力を状態に追加
        return self

    def model(self, prompt):
        self._state.append({'role': 'assistant', 'content': prompt})  # モデルの出力を状態に追加
        return self
    
    def system(self, prompt):
        self._state.append({'role': 'system', 'content': prompt})  # システムのプロンプトを状態に追加
        return self

    def reset(self):
        self._state = []  # 状態をリセット
        if self._system_prompt is not None:
            self.system(self._system_prompt)  # システムプロンプトがある場合は追加
        if self._few_shot_examples is not None:
            self.apply_turns(self._few_shot_examples, start_agent='user')  # Few-shot例を適用
        return self

    def apply_turns(self, turns: Iterable, start_agent: str):
        formatters = [self.model, self.user] if start_agent == 'model' else [self.user, self.model]
        formatters = itertools.cycle(formatters)  # エージェントを循環
        for fmt, turn in zip(formatters, turns):  # プロンプトのターンを適用
            fmt(turn)
        return self


# エージェントの定義
import re

class Llama3Agent:
    def __init__(self, system_prompt=None, few_shot_examples=None):
        self.formatter = Llama3Formatter(system_prompt=system_prompt, few_shot_examples=few_shot_examples)  # フォーマッタを初期化
        self.tokenizer = AutoTokenizer.from_pretrained(WEIGHTS_PATH)  # トークナイザーを初期化
        self.terminators = [self.tokenizer.eos_token_id, self.tokenizer.convert_tokens_to_ids("")]
        
        ### 元のモデルをロード
        self.model = AutoModelForCausalLM.from_pretrained(
            WEIGHTS_PATH,
            device_map="auto",  # デバイスの割り当てを自動で行う
            torch_dtype=torch.bfloat16  # モデルのデータ型を指定
        )

    def __call__(self, obs, *args):  # エージェントの呼び出し
        self._start_session(obs)  # セッションを開始
        prompt = self.formatter.get_dict()  # プロンプトを取得
        response = self._call_llm(prompt)  # LLMを呼び出して応答を取得
        response = self._parse_response(response, obs)  # 応答を解析
        return response

    def _start_session(self, obs: dict):
        raise NotImplementedError  # サブクラスで実装する必要がある

    def _call_llm(self, prompt, max_new_tokens=32):
        input_ids = self.tokenizer.apply_chat_template(
            prompt,  # プロンプトをトークンに変換
            add_generation_prompt=True,  # 生成プロンプトを追加
            return_tensors="pt",  # PyTorchテンソルとして返す
        ).to(self.model.device)  # モデルのデバイスに移動
        outputs = self.model.generate(
            input_ids=input_ids,  # エンコードしたプロンプト
            max_new_tokens=max_new_tokens,  # 最大のトークン数
            eos_token_id=self.terminators,  # 終了トークン
            do_sample=True,  # サンプリングを有効にする
            temperature=0.6,  # サンプリング温度
            top_p=0.9  # Top-pサンプリング
        )
        response = self.tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)  # 応答をデコード

        return response

    def _parse_keyword(self, response: str):
        match = re.search(r"(?<=\*\*)([^*]+)(?=\*\*)", response)  # 特殊なキーワードを抽出
        if match is None:
            keyword = ''
        else:
            keyword = match.group().lower()  # 小文字に変換
        return keyword

    def _parse_response(self, response: str, obs: dict):
        raise NotImplementedError  # サブクラスで実装する必要がある


def interleave_unequal(x, y):
    return [
        item for pair in itertools.zip_longest(x, y) for item in pair if item is not None
    ]


class Llama3QuestionerAgent(Llama3Agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  # 親クラスの初期化
        self.category_determined = False  # カテゴリが決定されたかどうか
        self.is_place = False  # 場所かどうか
        self.first_char_range = (0, 25)  # AからZまでのインデックス
        self.second_char_range = None  # 二文字目の範囲
        self.final_guess = None  # 最終的な推測

    def _start_session(self, obs):
        global guesses

        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user("Let's play 20 Questions. You are playing the role of the Questioner.")  # ゲーム開始メッセージ
        turns = interleave_unequal(obs.questions, obs.answers)  # ユーザーの質問と回答を交互に配置
        self.formatter.apply_turns(turns, start_agent='model')  # ターンを適用

        if not self.category_determined:
            self.formatter.user("Is it a place?")  # 最初の質問
        else:
            if self.second_char_range is None:
                mid_index = (self.first_char_range[0] + self.first_char_range[1]) // 2  # 中間インデックスを計算
                mid_char = chr(65 + mid_index)  # インデックスをアルファベットに変換（0 → A, 1 → B, ... 25 → Z）
                self.formatter.user(f"Does the keyword start with a letter before {mid_char}?")  # 中間文字より前かどうかを質問
            elif self.final_guess is None:
                mid_index = (self.second_char_range[0] + self.second_char_range[1]) // 2  # 二文字目の範囲の中間インデックス
                mid_char = chr(65 + mid_index)
                self.formatter.user(f"Does the second letter of the keyword come before {mid_char}?")  # 二文字目に関する質問
            else:
                self.formatter.user(f"Is the keyword **{self.final_guess}**?")  # 最終的な推測の質問

    def _parse_response(self, response: str, obs: dict):
        if not self.category_determined:
            answer = self._parse_keyword(response)  # 応答からキーワードを解析
            self.is_place = (answer == 'yes')  # 質問が「はい」の場合、場所であることを示す
            self.category_determined = True  # カテゴリが決定
            return "Is it a place?"  # 次の質問
        else:
            if self.second_char_range is None:
                answer = self._parse_keyword(response)  # 応答からキーワードを解析
                mid_index = (self.first_char_range[0] + self.first_char_range[1]) // 2  # 中間インデックスを計算
                if answer == 'yes':
                    self.first_char_range = (self.first_char_range[0], mid_index)  # 範囲を狭める
                else:
                    self.first_char_range = (mid_index + 1, self.first_char_range[1])  # 範囲を変更

                if self.first_char_range[0] == self.first_char_range[1]:  # 範囲が一つの文字に絞られた場合
                    self.second_char_range = (0, 25)  # 二文字目の範囲をリセット
                    return f"Does the keyword start with {chr(65 + self.first_char_range[0])}?"  # 絞られた文字で質問
                else:
                    mid_index = (self.first_char_range[0] + self.first_char_range[1]) // 2
                    mid_char = chr(65 + mid_index)
                    return f"Does the keyword start with a letter before {mid_char}?"  # 中間文字に基づき質問
            elif self.final_guess is None:
                answer = self._parse_keyword(response)  # 応答からキーワードを解析
                mid_index = (self.second_char_range[0] + self.second_char_range[1]) // 2
                if answer == 'yes':
                    self.second_char_range = (self.second_char_range[0], mid_index)  # 範囲を狭める
                else:
                    self.second_char_range = (mid_index + 1, self.second_char_range[1])  # 範囲を変更

                if self.second_char_range[0] == self.second_char_range[1]:  # 範囲が一つの文字に絞られた場合
                    first_char = chr(65 + self.first_char_range[0])
                    second_char = chr(65 + self.second_char_range[0])
                    self.final_guess = first_char + second_char  # 最終推測を更新
                    return f"Does the keyword start with {first_char}{second_char}?"  # 最終推測で質問
                else:
                    mid_index = (self.second_char_range[0] + self.second_char_range[1]) // 2
                    mid_char = chr(65 + mid_index)
                    return f"Does the second letter of the keyword come before {mid_char}?"  # 二文字目に関する質問
            else:
                answer = self._parse_keyword(response)  # 応答からキーワードを解析
                if answer == 'yes':
                    return f"The keyword is **{self.final_guess}**."  # 正しい推測
                else:
                    self.final_guess = None  # 推測をリセット
                    return "Let's continue guessing."  # ゲームを続ける


class Llama3AnswererAgent(Llama3Agent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)  # 親クラスの初期化

    def _start_session(self, obs):
        self.formatter.reset()  # フォーマッタをリセット
        self.formatter.user(f"Let's play 20 Questions. You are playing the role of the Answerer. The keyword is {obs.keyword} in the category {obs.category}.")  # ゲーム開始メッセージ
        turns = interleave_unequal(obs.questions, obs.answers)  # ユーザーの質問と回答を交互に配置
        self.formatter.apply_turns(turns, start_agent='user')  # ターンを適用
        self.formatter.user(f"The question is about the keyword {obs.keyword} in the category {obs.category}. Give yes-or-no answer and surround your answer with double asterisks, like **yes** or **no**.")  # 応答の形式を指示

    def _parse_response(self, response: str, obs: dict):
        answer = self._parse_keyword(response)  # 応答からキーワードを解析
        return 'yes' if 'yes' in answer else 'no'  # 「はい」または「いいえ」で返す


# エージェントの作成
system_prompt = "You are a very smart AI assistant designed to play the 20 Questions game. In this game, the Answerer thinks of a keyword and responds to yes-or-no questions by the Questioner. The keyword is a specific place or thing."

few_shot_examples = [
    "Let's play 20 Questions. You are playing the role of the Questioner. Please ask your first question.",
    "Is the category of the keyword place?", "**no**",
    "Is it a food?", "**yes** Now guess the keyword in the category things.",
    "**Veggie Burger**", "Correct.",  # Few-shot例の設定
]

# **重要:** エージェントをグローバルに定義して、必要なエージェントだけをロードします。
# 両方をロードすると、OOMになる可能性があります。
agent = None


def get_agent(name: str):
    global agent
    
    if agent is None and name == 'questioner':
        agent = Llama3QuestionerAgent(
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    elif agent is None and name == 'answerer':
        agent = Llama3AnswererAgent(
            system_prompt=system_prompt,
            few_shot_examples=few_shot_examples,
        )
    assert agent is not None, "Agent not initialized."  # エージェントが初期化されているか確認

    return agent


turnRound = 1  # ラウンド数を初期化
guesses = []  # 推測のリストを初期化

def agent_fn(obs, cfg):
    global turnRound
    global guesses

    if obs.turnType == "ask":  # 質問の場合
        if turnRound == 1:
            response = "Is it a place?"  # 最初の質問
        else:
            response = get_agent('questioner')(obs)  # 質問エージェントから応答を取得
    elif obs.turnType == "guess":  # 推測の場合
        response = get_agent('questioner')(obs)  # 質問エージェントから応答を取得
        turnRound += 1  # ラウンド数を増やす
        guesses.append(response)  # 推測をリストに追加
    elif obs.turnType == "answer":  # 回答の場合
        response = get_agent('answerer')(obs)  # 回答エージェントから応答を取得
        turnRound += 1  # ラウンド数を増やす
    if response is None or len(response) <= 1:  # 応答が無効な場合
        return "yes"  # デフォルトの「はい」を返す
    else:
        return response  # 有効な応答を返す

In [ ]:
!apt install pigz pv > /dev/null  # pigzとpvパッケージをインストールします
# pigzは高速なgzip互換の圧縮ツールで、pvはパイプの進行状況を表示するツールです
# インストールの出力は非表示に設定しています（> /dev/null）

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission . -C /kaggle/input/ llama-3/transformers/8b-chat-hf/1
# tarコマンドを使用して、submission.tar.gzという名前のアーカイブを作成します
# --use-compress-programオプションでpigzを指定し、圧縮を高速化しています
# -Cオプションでディレクトリを変更し、submissionとllama-3の内容をアーカイブに追加します
# pvを使用して進行状況を表示します

In [ ]:
%%bash
# 新しいディレクトリを作成します
# -pオプションは、必要に応じて親ディレクトリも作成します
mkdir -p /kaggle/working/submission